In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

from sklearn import cluster, metrics
from sklearn import manifold, decomposition

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from gensim.models.phrases import Phrases

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier


import time

import pickle

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(pickle.__version__)? (203225737.py, line 40)

In [ ]:
sample_posts = pd.read_csv("data/subsetTop30Tags2.csv", sep=";")
sample_posts.head()

In [ ]:
sample_posts.shape

In [ ]:
sample_posts = sample_posts.sample(5000)
sample_posts

## Classifier

In [2]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(sample_posts.FullPosts))

print(data_words[:3])

NameError: name 'sample_posts' is not defined

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=2,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=50000             # max number of uniq words
                            )
# vectorizer.fit(data_lemmatized)
# data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
# pickle.dump(data_vectorized, open('../Preprocessing/vectorizer.pkl','wb'))

In [ ]:
# data_vectorized = vectorizer.fit_transform(data_lemmatized)
# print(vectorizer.get_feature_names_out())

In [ ]:
# # Materialize the sparse data
# data_dense = data_vectorized.todense()

# # Compute Sparsicity = Percentage of Non-Zero cells
# print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [ ]:
X = sample_posts["FullPosts"]
y = sample_posts["Tags"]

In [ ]:
X = vectorizer.fit_transform(X)

In [ ]:
pickle.dump(vectorizer, open('../Preprocessing/vectorizer.pkl','wb'))

In [ ]:
# Multilabel binarizer for targets
# multilabel_binarizer = MultiLabelBinarizer()
# multilabel_binarizer.fit(y)
# y_binarized = multilabel_binarizer.transform(y)

In [ ]:
# X = vectorizer.fit_transform(X)

In [ ]:
# Split from the loaded dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
print("X_train shape : {}".format(X_train.shape))
print("X_test shape : {}".format(X_test.shape))
print("y_train shape : {}".format(y_train.shape))
print("y_test shape : {}".format(y_test.shape))

In [ ]:
def lsa_reduction(X_train, X_test, n_comp=120):
    svd = TruncatedSVD(n_components=n_comp)
    normalizer = Normalizer()
    
    lsa_pipe = Pipeline([('svd', svd),
                        ('normalize', normalizer)]).fit(X_train)
    
    train_reduced = lsa_pipe.transform(X_train)
    test_reduced = lsa_pipe.transform(X_test)
    return train_reduced, test_reduced

In [ ]:
# Créer une instance de CountVectorizer pour la vectorisation de texte
# vect = CountVectorizer()

# Créer une instance de SVC pour l'apprentissage multi-étiquettes
svc = SVC(kernel='linear')
lsvc = LinearSVC(random_state=0)

# Créer une instance de OneVsRestClassifier pour effectuer la classification multi-étiquettes
model_OVR = OneVsRestClassifier(svc)

# Créer un pipeline qui combine la vectorisation de texte et la classification multi-étiquettes
# model_OVR = Pipeline([('vect', vect), ('ovr', ovr)])

# Entraîner le pipeline sur les données d'entraînement
# X_train = vectorizer.fit_transform(X_train)
model_OVR.fit(X_train, y_train)

# Évaluer le pipeline sur les données de test
# predict on test data
# X_test = vectorizer.transform(X_test)
y_pred = model_OVR.predict(X_test)
print(y_pred)
print(y_pred.shape)

In [ ]:
pickle.dump(model_OVR, open('../ModelsAPI/model_OVR.pkl','wb'))

In [ ]:
test = ["Django supports Python. If you're under Linux and want to check the Python version you're using, run python -V from the command line If you want to check the Django version, open a Python console and type"]
# test_words = list(sent_to_words(test))
# test_words = lemmatization(test_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# test_words = count_vect.transform(test_words)
pred = model_OVR.predict(vectorizer.transform(test))
print(pred)

In [ ]:
# pipeline = Pipeline([('vect', vectorizer, 'clf', model_OVR)])

# # train classifier
# pipeline.fit(X_train)

# evaluate all steps on test set
# predicted = pipeline.predict(X_test)